In [1]:
import pandas as pd
from daq.interfacedb import setup_lbnl

import re 

In [2]:
# %load C:\Users\Walter\Desktop\tools\template\plot_template.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns

plt.rcParams['figure.figsize'] = [10,7]
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['legend.loc'] = 'lower left'
%matplotlib inline

In [3]:
db,dbname = setup_lbnl()

In [4]:
mea_list = db.get_measurements_list(dbname)

print(mea_list)

[u'1045', u'1065', u'1125', u'12458', u'12541', u'12614', u'12635', u'12743', u'13881', u'145', u'165', u'205', u'245', u'2525', u'2545', u'2565', u'2585', u'2605', u'2625', u'2645', u'265', u'2665', u'2685', u'2705', u'2725', u'2745', u'2865', u'2885', u'2905', u'2925', u'2945', u'2965', u'2985', u'305', u'30934', u'33336', u'33400', u'34222', u'34242', u'345', u'385', u'405', u'425', u'42808', u'445', u'465', u'485', u'505', u'525', u'545', u'565', u'625', u'645', u'665', u'685', u'705', u'725', u'745', u'765', u'785', u'805', u'825', u'845', u'865', u'905', u'925', u'92753', u'945', u'965', u'CAISO_NORTH', u'CAISO_NORTH_historical', u'CAISO_ZP26', u'CAISO_ZP26_historical', u'R2_solar_train', u'R2_solar_train_ds', u'R2_solar_valid', u'R2_solar_valid_ds', u'RMSE_solar_train', u'RMSE_solar_train_ds', u'RMSE_solar_valid', u'RMSE_solar_valid_ds', u'air_temp_set_1', u'air_temp_set_2', u'alc_watchdog_59', u'alc_watchdog_76', u'altimeter_set_1d', u'aru1_hwr_temp', u'aru1_hws_fr_gpm', u'aru1

In [5]:
start_time = '2018-01-01'
final_time = '2021-01-01'

## Version 1

In [10]:
# Site weather
site_weather = ['air_temp_set_'+str(i) for i in range(1,3)] + \
               ['dew_point_temperature_set_1d','relative_humidity_set_1'] + \
               ['solar_radiation_set_1']

In [7]:
# Outside Air
oa_flow_rate_re = re.compile("rtu_.*_oa_flow_tn")
oa_flow_rate = list(filter(oa_flow_rate_re.match, mea_list))

oa_damper_position_re = re.compile("rtu_.*_oadmpr_pct")
oa_damper_position = list(filter(oa_damper_position_re.match, mea_list))

oa = oa_flow_rate + oa_damper_position

In [11]:
# Zone: temp.
zone_temp_exterior_re = re.compile("zone_.*_temp")
zone_temp_exterior = list(filter(zone_temp_exterior_re.match, mea_list))

zone_temp_interior_re = re.compile("cerc_.*")
zone_temp_interior = list(filter(zone_temp_interior_re.match, mea_list))

In [12]:
# Zone: temp. setpoint
zone_temp_sp_h_re = re.compile("zone_.*_heating_sp")
zone_temp_sp_h = list(filter(zone_temp_sp_h_re.match, mea_list))

zone_temp_sp_c_re = re.compile("zone_.*_cooling_sp")
zone_temp_sp_c = list(filter(zone_temp_sp_c_re.match, mea_list))

In [13]:
# RTU: Supply air temp.
rtu_sa_t_sp_re = re.compile("rtu_.*_sat_sp_tn")
rtu_sa_t_sp = list(filter(rtu_sa_t_sp_re.match, mea_list))

rtu_sa_t_re = re.compile("rtu_.*_sa_temp")
rtu_sa_t = list(filter(rtu_sa_t_re.match, mea_list))

In [14]:
# RTU: Supply air slow rate
rtu_sa_p_sp_re = re.compile("rtu_.*_pa_static_stpt_tn")
rtu_sa_p_sp = list(filter(rtu_sa_p_sp_re.match, mea_list))

rtu_sa_fr_re = re.compile("rtu_.*_fltrd_sa_flow_tn")
rtu_sa_fr = list(filter(rtu_sa_fr_re.match, mea_list))

In [15]:
# UFT: supply air temp.
uft_hw_valve_re = re.compile("zone_.*_hw_valve")
uft_hw_valve = list(filter(uft_hw_valve_re.match, mea_list))

hp_hws_temp_re = re.compile(".*_hws_temp")
hp_hws_temp = list(filter(hp_hws_temp_re.match, mea_list))

In [16]:
# UFT: supply air flow rate
uft_fan_spd_re = re.compile("zone_.*_fan_spd")
uft_fan_spd = list(filter(uft_fan_spd_re.match, mea_list))

In [26]:
start_time = '2018-01-01'
final_time = '2021-01-01'
path_to_file = 'data/na/2020'

In [ ]:
def download_data(data_list,file_name,freq='30min',start_time = '2018-01-01',final_time = '2020-01-01'):
    data = pd.DataFrame()
    for field in data_list:
        data_single = db.get_data(field,dbname,start_time,final_time).resample(freq).mean()
        data = pd.concat([data,data_single], axis=1)
    data.to_csv('data/{}.csv'.format(file_name))

In [28]:
def download_data_raw(data_list,file_name,path_to_file,start_time = start_time,final_time = final_time):
    data = pd.DataFrame()
    for field in data_list:
        data_single = db.get_data(field,dbname,start_time,final_time)
        data = pd.concat([data,data_single], axis=1)
    data.to_csv('{}/{}_raw.csv'.format(path_to_file ,file_name))

In [20]:
path_to_file = 'data/na/more_rtu_data_18to20'

In [20]:
list_data_list = [site_weather, zone_temp_exterior, zone_temp_interior,
                  zone_temp_sp_h, zone_temp_sp_c,
                  rtu_sa_t_sp, rtu_sa_t, oa, economizer, 
                  rtu_sa_p_sp, rtu_sa_fr,
                  uft_hw_valve, hp_hws_temp, uft_fan_spd]
list_file_name = ['site_weather', 'zone_temp_exterior', 'zone_temp_interior',
                  'zone_temp_sp_h', 'zone_temp_sp_c',
                  'rtu_sa_t_sp', 'rtu_sa_t', 'outside_air', 'economizer',
                  'rtu_sa_p_sp', 'rtu_sa_fr',
                  'uft_hw_valve', 'hp_hws_temp', 'uft_fan_spd']

for data_list, file_name in zip(list_data_list,list_file_name):
    download_data_raw(data_list,file_name)

04/22/2021 04:28:58 PM	daq.daq.interfacedb INFO	Data for air_temp_set_1 retrieved from database dhblum.
04/22/2021 04:28:58 PM	daq.daq.interfacedb INFO	Data for air_temp_set_2 retrieved from database dhblum.
04/22/2021 04:28:59 PM	daq.daq.interfacedb INFO	Data for dew_point_temperature_set_1d retrieved from database dhblum.
04/22/2021 04:29:00 PM	daq.daq.interfacedb INFO	Data for relative_humidity_set_1 retrieved from database dhblum.
04/22/2021 04:29:01 PM	daq.daq.interfacedb INFO	Data for solar_radiation_set_1 retrieved from database dhblum.
04/22/2021 04:29:07 PM	daq.daq.interfacedb INFO	Data for zone_016_temp retrieved from database dhblum.
04/22/2021 04:29:11 PM	daq.daq.interfacedb INFO	Data for zone_017_temp retrieved from database dhblum.
04/22/2021 04:29:16 PM	daq.daq.interfacedb INFO	Data for zone_018_temp retrieved from database dhblum.
04/22/2021 04:29:21 PM	daq.daq.interfacedb INFO	Data for zone_019_temp retrieved from database dhblum.
04/22/2021 04:29:26 PM	daq.daq.interfa

04/22/2021 04:33:31 PM	daq.daq.interfacedb INFO	Data for zone_024_heating_sp retrieved from database dhblum.
04/22/2021 04:33:32 PM	daq.daq.interfacedb INFO	Data for zone_025_heating_sp retrieved from database dhblum.
04/22/2021 04:33:33 PM	daq.daq.interfacedb INFO	Data for zone_026_heating_sp retrieved from database dhblum.
04/22/2021 04:33:34 PM	daq.daq.interfacedb INFO	Data for zone_027_heating_sp retrieved from database dhblum.
04/22/2021 04:33:38 PM	daq.daq.interfacedb INFO	Data for zone_028_heating_sp retrieved from database dhblum.
04/22/2021 04:33:39 PM	daq.daq.interfacedb INFO	Data for zone_030_heating_sp retrieved from database dhblum.
04/22/2021 04:33:40 PM	daq.daq.interfacedb INFO	Data for zone_032_heating_sp retrieved from database dhblum.
04/22/2021 04:33:44 PM	daq.daq.interfacedb INFO	Data for zone_033_heating_sp retrieved from database dhblum.
04/22/2021 04:33:45 PM	daq.daq.interfacedb INFO	Data for zone_035_heating_sp retrieved from database dhblum.
04/22/2021 04:33:46

04/22/2021 04:35:48 PM	daq.daq.interfacedb INFO	Data for zone_053_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:49 PM	daq.daq.interfacedb INFO	Data for zone_054_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:50 PM	daq.daq.interfacedb INFO	Data for zone_055_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:51 PM	daq.daq.interfacedb INFO	Data for zone_056_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:52 PM	daq.daq.interfacedb INFO	Data for zone_057_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:55 PM	daq.daq.interfacedb INFO	Data for zone_058_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:57 PM	daq.daq.interfacedb INFO	Data for zone_059_cooling_sp retrieved from database dhblum.
04/22/2021 04:35:58 PM	daq.daq.interfacedb INFO	Data for zone_061_cooling_sp retrieved from database dhblum.
04/22/2021 04:36:02 PM	daq.daq.interfacedb INFO	Data for zone_062_cooling_sp retrieved from database dhblum.
04/22/2021 04:36:03

04/22/2021 04:41:00 PM	daq.daq.interfacedb INFO	Data for zone_070_hw_valve retrieved from database dhblum.
04/22/2021 04:41:04 PM	daq.daq.interfacedb INFO	Data for zone_071_hw_valve retrieved from database dhblum.
04/22/2021 04:41:09 PM	daq.daq.interfacedb INFO	Data for zone_072_hw_valve retrieved from database dhblum.
04/22/2021 04:41:19 PM	daq.daq.interfacedb INFO	Data for aru1_hws_temp retrieved from database dhblum.
04/22/2021 04:41:24 PM	daq.daq.interfacedb INFO	Data for aru_001_hws_temp retrieved from database dhblum.
04/22/2021 04:41:29 PM	daq.daq.interfacedb INFO	Data for hp_hws_temp retrieved from database dhblum.
04/22/2021 04:41:36 PM	daq.daq.interfacedb INFO	Data for zone_016_fan_spd retrieved from database dhblum.
04/22/2021 04:41:40 PM	daq.daq.interfacedb INFO	Data for zone_017_fan_spd retrieved from database dhblum.
04/22/2021 04:41:45 PM	daq.daq.interfacedb INFO	Data for zone_018_fan_spd retrieved from database dhblum.
04/22/2021 04:41:49 PM	daq.daq.interfacedb INFO	Dat

In [ ]:
# zone number mapping
zone_list_2_north = ['016','017','018','021','022','023','024','025','026','027','028']
zone_list_2_south = ['035','036','037','038','039','040','041','042','043','044']
zone_list_1_north = ['046','047','048','051','052','053','054','055','056','057', '058']
zone_list_1_south = ['063','064','065','066','067','068','069','070','071','072']
zone_list_core = ['019', '030', '032', '033', '045', '059', '061', '062', '049']

## Version 2 - after discussion with Dave

In [29]:
start_time = '2018-01-01'
final_time = '2021-01-01'

path_to_file = 'data/version2'

In [30]:
# Economizer
economizer_stpt_re = re.compile("rtu_.*_econ_stpt_tn")
economizer_stpt = list(filter(economizer_re.match, mea_list))

oa_flow_rate_re = re.compile("rtu_.*_oa_flow_tn")
oa_flow_rate = list(filter(oa_flow_rate_re.match, mea_list))

oa_damper_position_re = re.compile("rtu_.*_oadmpr_pct")
oa_damper_position = list(filter(oa_damper_position_re.match, mea_list))

economizer = economizer_stpt + oa_flow_rate + oa_damper_position
economizer

[u'rtu_001_econ_stpt_tn',
 u'rtu_002_econ_stpt_tn',
 u'rtu_003_econ_stpt_tn',
 u'rtu_004_econ_stpt_tn',
 u'rtu_001_oa_flow_tn',
 u'rtu_002_oa_flow_tn',
 u'rtu_003_oa_flow_tn',
 u'rtu_004_oa_flow_tn',
 u'rtu_001_oadmpr_pct',
 u'rtu_002_oadmpr_pct',
 u'rtu_003_oadmpr_pct',
 u'rtu_004_oadmpr_pct']

In [21]:
# air temperature in rtu
ra_temp_re = re.compile("rtu_.*_ra_temp")
ra_temp = list(filter(ra_temp_re.match, mea_list))

sa_temp_re = re.compile("rtu_.*_sa_temp")
sa_temp = list(filter(sa_temp_re.match, mea_list))

ma_temp_re = re.compile("rtu_.*_ma_temp")
ma_temp = list(filter(ma_temp_re.match, mea_list))

oa_temp_re = re.compile("rtu_.*_oa_temp")
oa_temp = list(filter(oa_temp_re.match, mea_list))

rtu_temp = ra_temp + sa_temp + ma_temp + oa_temp
rtu_temp

[u'rtu_001_ra_temp',
 u'rtu_002_ra_temp',
 u'rtu_003_ra_temp',
 u'rtu_004_ra_temp',
 u'rtu_001_sa_temp',
 u'rtu_002_sa_temp',
 u'rtu_003_sa_temp',
 u'rtu_004_sa_temp',
 u'rtu_001_ma_temp',
 u'rtu_002_ma_temp',
 u'rtu_003_ma_temp',
 u'rtu_004_ma_temp',
 u'rtu_001_oa_temp',
 u'rtu_002_oa_temp',
 u'rtu_003_oa_temp',
 u'rtu_004_oa_temp']

In [22]:
# RTU fan speed
rtu_supply_fan_speed_re = re.compile("rtu_.*_sf_vfd_spd_fbk_tn")
rtu_supply_fan_speed = list(filter(rtu_supply_fan_speed_re.match, mea_list))

rtu_return_fan_speed_re = re.compile("rtu_.*_rf_vfd_spd_fbk_tn")
rtu_return_fan_speed = list(filter(rtu_return_fan_speed_re.match, mea_list))

rtu_fan = rtu_supply_fan_speed + rtu_return_fan_speed
rtu_fan

[u'rtu_001_sf_vfd_spd_fbk_tn',
 u'rtu_002_sf_vfd_spd_fbk_tn',
 u'rtu_003_sf_vfd_spd_fbk_tn',
 u'rtu_004_sf_vfd_spd_fbk_tn',
 u'rtu_001_rf_vfd_spd_fbk_tn',
 u'rtu_002_rf_vfd_spd_fbk_tn',
 u'rtu_003_rf_vfd_spd_fbk_tn',
 u'rtu_004_rf_vfd_spd_fbk_tn']

In [23]:
# RTU plenum pressure
rtu_lvl_plenum_press_re = re.compile("rtu_.*_fltrd_gnd_lvl_plenum_press_tn")
rtu_lvl_plenum_press = list(filter(rtu_lvl_plenum_press_re.match, mea_list))

rtu_lv2_plenum_press_re = re.compile("rtu_.*_fltrd_lvl2_plenum_press_tn")
rtu_lv2_plenum_press = list(filter(rtu_lv2_plenum_press_re.match, mea_list))

rtu_plenum_pressure = rtu_lvl_plenum_press + rtu_lv2_plenum_press
rtu_plenum_pressure

[u'rtu_001_fltrd_gnd_lvl_plenum_press_tn',
 u'rtu_002_fltrd_gnd_lvl_plenum_press_tn',
 u'rtu_003_fltrd_gnd_lvl_plenum_press_tn',
 u'rtu_004_fltrd_gnd_lvl_plenum_press_tn',
 u'rtu_001_fltrd_lvl2_plenum_press_tn',
 u'rtu_002_fltrd_lvl2_plenum_press_tn',
 u'rtu_003_fltrd_lvl2_plenum_press_tn',
 u'rtu_004_fltrd_lvl2_plenum_press_tn']

In [32]:
# air source heat pump
ashp_re = re.compile("aru_.*")
ashp = list(filter(ashp_re.match, mea_list))

ashp

[u'aru_001_cwr_temp',
 u'aru_001_cws_fr_gpm',
 u'aru_001_cws_temp',
 u'aru_001_hwr_temp',
 u'aru_001_hws_fr_gpm',
 u'aru_001_hws_temp',
 u'aru_001_power_mbtuph']

In [33]:
list_data_list = [economizer, rtu_temp, rtu_fan, rtu_plenum_pressure, ashp]
list_file_name = ['economizer', 'rtu temperature', 'rtu fan speed', 'rtu plenum pressure', 'ashp for terminal reheat']

for data_list, file_name in zip(list_data_list,list_file_name):
    download_data_raw(data_list, file_name, path_to_file, start_time=start_time, final_time=final_time)

09/04/2021 01:30:47 PM	daq.daq.interfacedb INFO	Data for rtu_001_econ_stpt_tn retrieved from database dhblum.
09/04/2021 01:30:58 PM	daq.daq.interfacedb INFO	Data for rtu_002_econ_stpt_tn retrieved from database dhblum.
09/04/2021 01:31:10 PM	daq.daq.interfacedb INFO	Data for rtu_003_econ_stpt_tn retrieved from database dhblum.
09/04/2021 01:31:21 PM	daq.daq.interfacedb INFO	Data for rtu_004_econ_stpt_tn retrieved from database dhblum.
09/04/2021 01:31:25 PM	daq.daq.interfacedb INFO	Data for rtu_001_oa_flow_tn retrieved from database dhblum.
09/04/2021 01:31:29 PM	daq.daq.interfacedb INFO	Data for rtu_002_oa_flow_tn retrieved from database dhblum.
09/04/2021 01:31:32 PM	daq.daq.interfacedb INFO	Data for rtu_003_oa_flow_tn retrieved from database dhblum.
09/04/2021 01:31:36 PM	daq.daq.interfacedb INFO	Data for rtu_004_oa_flow_tn retrieved from database dhblum.
09/04/2021 01:31:48 PM	daq.daq.interfacedb INFO	Data for rtu_001_oadmpr_pct retrieved from database dhblum.
09/04/2021 01:32:00 

In [ ]:
import influxdb
print(influxdb.__version__)